In [2]:

import snscrape.modules.twitter as sntwitter
import pandas as pd
import time
import glob
import os
import time
import datetime

In [3]:
#!pip3 install snscrape
#!pip install pyarrow
#!pip install fastparquet

In [4]:

#to get the current working directory
# directory = os.getcwd()

# print(directory)
# for name in glob.glob(r'{}\data_minutos_dataframe_shifts_*.parquet'.format(directory)):
#     print(name)
#     pd.read_parquet(name)

In [5]:
#pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

In [6]:
# Likes: tweet.likeCount
# Retweets: tweet.retweetCount
# Replies: tweet.replyCount
# Quotes: tweet.quoteCount
#  or sntwitter.TwitterSearchScraper('#doge since:2021-01-31 until:2021-12-20').get_items()
#2021-12-20

In [7]:
def GetTweets(quantityToAnalize, minimunLikeCount,sinceDate,untilDate):

#Este código demora 4.25 segundos para scrappear 2000 tweets
# lista donde se guardaran los tweets y su info
    tweets_list= []
    t0 = time.time()
# Scrapeando los tweets que contengan #doge o que contengan #dogecoin, con since y until se le indica el período.
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#dogecoin since:{since} until:{until}'.format(since = sinceDate, until=untilDate)).get_items()or sntwitter.TwitterSearchScraper('#doge since:2021-01-31 until:2021-12-20').get_items()):
        if i>quantityToAnalize:
            break
        if tweet.likeCount > minimunLikeCount:
            tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount ])

    t1=time.time()
    print("time elapsed: {:.1f} min".format((t1-t0)/60))
    result_dataframe = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])
    return result_dataframe

In [8]:
# date_datetime= pd.to_datetime(dataframe["Datetime"].iloc[1])
# date_datetime_end=pd.to_datetime(dataframe["Datetime"].iloc[-1])
# fecha_inicial = "{year}-{month}-{day}".format(year =date_datetime.year,month=date_datetime.month,day=date_datetime.day)  
# fecha_final = "{year}-{month}-{day}".format(year =date_datetime_end.year,month=date_datetime_end.month,day=date_datetime_end.day)  

In [9]:
#Este código no funciona como yo espero. Entonces voy a dividir la funcionalidad en dos

# def BreakOutScrape(totalQuantityToAnalize, blocks):
#     round_partition = round(totalQuantityToAnalize / blocks)
   
#     for i in range(blocks):
#         #t0 = time.time()
#         if i == 0:   
#             dataframe = GetTweets(round_partition,10,'2021-01-01','2021-12-20')
#             fecha_mayor_datetime=pd.to_datetime(dataframe["Datetime"].iloc[-1])
#             fecha_mayor = "{year}-{month}-{day}".format(year =fecha_mayor_datetime.year,month=fecha_mayor_datetime.month,day=fecha_mayor_datetime.day)  
#             dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=i))
        
#         else:
#             directory = os.getcwd()
#             for name in glob.glob(r'{}\new_scraped_tweets_*.parquet'.format(directory)):
               
#                 dataframe_actual = pd.read_parquet(name)
#                 fecha_actual_datetime = pd.to_datetime(dataframe_actual["Datetime"].iloc[-1])
#                 fecha_actual = "{year}-{month}-{day}".format(year =fecha_actual_datetime.year,month=fecha_actual_datetime.month,day=fecha_actual_datetime.day)
#                 print(name)
#                 if(fecha_actual > fecha_mayor):
#                     fecha_mayor = fecha_actual
                    
#             dataframe = GetTweets(round_partition,10,fecha_mayor,'2021-12-20')
#             return dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=i))
           # t1=time.time()
            
    

Bien, la libreria comienza a buscar tweets del mas nuevo al mas viejo. Por eso nunca se llega a la fecha menor de la busqueda.

In [10]:
#Aca voy a hacer una segunda función que va a tomar lo que haya scrapeado y va a reanudar el scrapeo 
#Erroneamente crei que el scrapeo comienza de la fecha más antigua a la actualidad pero parece ser que se da en el orden inverso
#Que el paginado lista de la más actual a lo más antiguo. Por lo tanto para reanudar debería tomar la fecha menor y no la mayor

def GetMinDate(dataframe_actual):
        fecha_menor = pd.to_datetime(dataframe_actual["Datetime"].iloc[-1])+datetime.timedelta(days=1) #Se agrega un dia a la fecha menor para solapar los dfs y no perder info
        fecha_menor = "{year}-{month}-{day}".format(year =fecha_menor.year,month=fecha_menor.month,day=fecha_menor.day)
        
#         for i,row in dataframe_actual.iterrows():
#             fecha_actual_datetime = pd.to_datetime(row[0])
#             fecha_actual = "{year}-{month}-{day}".format(year =fecha_actual_datetime.year,month=fecha_actual_datetime.month,day=fecha_actual_datetime.day)
        
#             if(fecha_actual > fecha_menor):E
#                 fecha_menor = fecha_actual
        return fecha_menor 
            

In [11]:
#Esta función va a dividir la cantidad de tweets a scrapear en bloques. 
def BreakOutScrape(totalQuantityToAnalize, blocks):
    round_partition = round(totalQuantityToAnalize / blocks)
   
    for i in range(blocks):
        #t0 = time.time()
        if i == 0:   
            dataframe = GetTweets(round_partition,100,'2021-01-01','2021-12-20')
            #Funcion para buscar la fecha MENOR del dataframe
            #fecha_mayor_datetime=pd.to_datetime(dataframe["Datetime"].iloc[-1])
            #fecha_mayor = "{year}-{month}-{day}".format(year =fecha_mayor_datetime.year,month=fecha_mayor_datetime.month,day=fecha_mayor_datetime.day)  
            dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=i))
            fecha_menor = GetMinDate(dataframe)
            print(fecha_menor)
        
        else:
            
            dataframe = GetTweets(round_partition,100,'2021-01-01',fecha_menor)
            fecha_menor = GetMinDate(dataframe)
            print(fecha_menor)
            dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=i))
            print("Error raised")
                
            #Funcion para buscar la fecha MENOR del dataframe
           # t1=time.time()
            
    

In [12]:
#Esta función va a dividir la cantidad de tweets a scrapear en bloques. 
def BreakOutScrapeV2(totalQuantityToAnalize, blocks):
    round_partition = round(totalQuantityToAnalize / blocks)
    count=0
    while count<blocks:
        #t0 = time.time()
        if count == 0:
            try:
                dataframe = GetTweets(round_partition,100,'2021-01-01','2021-12-20')
                #Funcion para buscar la fecha MENOR del dataframe
                #fecha_mayor_datetime=pd.to_datetime(dataframe["Datetime"].iloc[-1])
                #fecha_mayor = "{year}-{month}-{day}".format(year =fecha_mayor_datetime.year,month=fecha_mayor_datetime.month,day=fecha_mayor_datetime.day)  
                dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(count))
                fecha_menor = GetMinDate(dataframe)
                print(fecha_menor)
                count+=1
            except:
                print("Error raised")
                time.sleep(300)
                continue 
        
        else:
            try:
                dataframe = GetTweets(round_partition,100,'2021-01-01',fecha_menor)
                fecha_menor = GetMinDate(dataframe)
                print(fecha_menor)
                dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(count))
                count+=1
            except:
                print("Error raised")
                continue 
            #Funcion para buscar la fecha MENOR del dataframe
           # t1=time.time()
            
    

In [13]:
def GetMaxFileNumber():
    import re
    directory = os.getcwd()
    numeracion=[]
    for name in glob.glob(directory+r"\new_scraped_tweets_*.parquet"):
        txt = name
        x = re.findall('[0-9]+', txt)
        numeracion.append(int(x[0]))
    numero_mayor = max(numeracion)
    return numero_mayor

In [14]:
def ResumeScraping(dataframe_stacked,totalQuantityToAnalize,blocks):
    round_partition = round(totalQuantityToAnalize / blocks)
    
    max_number = GetMaxFileNumber()
    
    for i in range(blocks):
        if i==0:
            fecha_menor = GetMinDate(dataframe_stacked)
            dataframe = GetTweets(round_partition,100,'2021-01-01',fecha_menor)
            fecha_menor = GetMinDate(dataframe)
            print(fecha_menor)
            number_for_name = i + max_number + 1
            dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=number_for_name))
            
        else:
            number_for_name = i + max_number + 1
            dataframe = GetTweets(round_partition,100,'2021-01-01',fecha_menor)
            fecha_menor = GetMinDate(dataframe)
            print(fecha_menor)
            dataframe.to_parquet("new_scraped_tweets_{numero}.parquet".format(numero=number_for_name))
    return True

In [15]:
#Aca va la función que toma los archivos de BreakOutScrape y genera un solo dataframe sin duplicados. 
def StackDataframes(nombre_partron_de_archivos):
    directory = os.getcwd()
    lista_dfs=[]
    for name in glob.glob(directory+nombre_partron_de_archivos):
        print(name)
        dataframe_actual = pd.read_parquet(name)
        lista_dfs.append(dataframe_actual)
        
    dafatame_vertical_stack = pd.concat(lista_dfs, axis=0)
    percentage = dafatame_vertical_stack.duplicated().sum()/len(dafatame_vertical_stack)
    print("Percentage of duplicated rows:",percentage)
    print(len(dafatame_vertical_stack))
    dafatame_vertical_stack_no_duplicated=dafatame_vertical_stack.drop_duplicates(subset="Tweet Id")
    print(len(dafatame_vertical_stack_no_duplicated))
    return dafatame_vertical_stack_no_duplicated

In [16]:
#Aca va a haber una función para retomar el scrape cuando se corte por algun motivo BreakOutScrape. Para esto va a leer el dataframe de salida que genera este codigo, la fecha de inicio objetivo, las particiones y continuará scrapeando

In [17]:
def EraseFiles(nombre_partron_de_archivos):
    directory = os.getcwd()
  
    for name in glob.glob(directory+nombre_partron_de_archivos):
          os.remove(name)
        

El argumento de la funcion de scraping de la libreria snscrape solo toma como parametro de fecha el formato yyyy-mm-dd por lo tanto no es posible cortar los datos a scrapear
en bloque sin solapar los "extremos". Por esto le sumo un dia mas a la funcion porque sino hay datos que pierdo entonces prefiero ser redundante con los dias en los extremos
de cada bloque y no perder informacion. Para los tests con 10.000 tweets a analizar en 4 bloques el solapamiento fue del 75%. Para 40.000 en 4 bloques fue de 27%. Entonces el solapamiento sera menor cuanto mayor sea la cantidad de tweets a analizar y menor la cantidad de bloques. Para 40.000 y 2 bloques el porcentaje fue de 0.9%. Hay que comprobar que el tamaño de bloque 20.000 arroje esos valores. Con 27% de solapamiento tardé 6 minutos por día. Con 0.9% de solapamiento tardé 4.5 minutos por d

In [18]:
BreakOutScrape(80000,8)

time elapsed: 9.0 min
2021-12-18


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23dogecoin+since%3A2021-01-01+until%3A2021-12-18&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAwKqBsJ3w6ygWjIC94fCSye0oEnEV7J1zFYCJehgEVVNFUjUBFeYKFQAA&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel: ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/search/adaptive.json?include_profile_interstiti

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23dogecoin+since%3A2021-01-01+until%3A2021-12-18&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAwKqBsJ3w6ygWjIC94fCSye0oEnEV7J1zFYCJehgEVVNFUjUBFeYKFQAA&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [27]:
 dataframe_stacked = StackDataframes(r"\new_scraped_tweets_*.parquet")

C:\Users\Claudio\Desktop\Archivos\DH -Data Science\NLPK\new_scraped_tweets_0.parquet
C:\Users\Claudio\Desktop\Archivos\DH -Data Science\NLPK\new_scraped_tweets_1.parquet
C:\Users\Claudio\Desktop\Archivos\DH -Data Science\NLPK\new_scraped_tweets_2.parquet
C:\Users\Claudio\Desktop\Archivos\DH -Data Science\NLPK\new_scraped_tweets_3.parquet
Percentage of duplicated rows: 0.03526093088857546
1418
1367


In [29]:
GetMinDate(dataframe_stacked)

'2021-12-5'

In [20]:
ResumeScraping(dataframe_stacked,60000,3)

time elapsed: 18.1 min
2021-12-12
time elapsed: 19.2 min
2021-12-7
time elapsed: 13.5 min
2021-12-5


In [35]:
#EraseFiles(r"\new_scraped_tweets_*.parquet")

In [ ]:
directory = os.getcwd()
    lista_dfs=[]
    for name in glob.glob(directory+nombre_partron_de_archivos):
        print(name)
        dataframe_actual = pd.read_parquet(name)
        lista_dfs.append(dataframe_actual)

In [14]:
fecha_menor = GetMinDate(dataframe_stacked)
fecha_menor

NameError: name 'dataframe_stacked' is not defined

In [49]:
import datetime

In [52]:
pd.to_datetime(fecha_menor)+datetime.timedelta(days=1)

Timestamp('2021-12-13 00:00:00')

In [45]:
lista_dfs[1]

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-17 23:55:20+00:00,1471992441385402370,#HODL on #dogearmy 💯💯💯🔥🔥🔥💪🏼💪🏼💪🏼 we gonna be al...,topherAlba26,122,44,24,0
1,2021-12-17 23:53:31+00:00,1471991982440562702,Something for you to to bless your phone backg...,_mintydoge,101,35,16,0
2,2021-12-17 23:09:55+00:00,1471981010871103497,Ok then\n#doge #dogecoin (middle class college...,dogeofficialceo,112,16,12,0
3,2021-12-17 23:05:43+00:00,1471979954715906071,#Dogecoin #doge $doge 😍❤️ https://t.co/VfNMjOeKKH,dogesuppporteam,201,39,23,1
4,2021-12-17 22:42:28+00:00,1471974101182988290,today i realized that i reached 30k followers ...,BigImpactHumans,183,17,37,1
...,...,...,...,...,...,...,...,...
83,2021-12-16 20:31:03+00:00,1471578641943330816,Who was the first celebrity you saw support #D...,Cocochaneladair,187,18,66,4
84,2021-12-16 20:02:51+00:00,1471571544031649793,YTD performance of #Dogecoin. Still looks goo...,realDogecoinOG,147,24,15,3
85,2021-12-16 19:38:05+00:00,1471565311417540610,DOGEEX IS NOW LIVE! \n\nJOIN THE FIRST DECENTR...,MiniDOGEToken,234,120,69,13
86,2021-12-16 19:12:07+00:00,1471558778323550209,Are you guys gonna dump #Dogecoin when we hit ...,calic415,354,44,46,1


In [34]:
dataframe_stacked

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-19 23:58:11+00:00,1472717932916576259,Keep up the good work Elon @elonmusk ⚒️🔥 Show ...,DogecoinNorway,607,100,38,6
1,2021-12-19 23:36:20+00:00,1472712433869279232,Congrats on 900k followers @BillyM2k! (Billy c...,itsALLrisky,313,47,19,1
2,2021-12-19 23:11:11+00:00,1472706104991879171,#Dogecoin will make millionaires,itsALLrisky,808,109,81,6
3,2021-12-19 23:08:01+00:00,1472705306182492164,Congratulations to #DogeBonk for becoming the ...,kcsdaily,428,220,94,11
4,2021-12-19 22:40:10+00:00,1472698297521348614,#SORRYELON WE MADE IT TO OUTER SPACE BEFORE #D...,bonktovi,106,43,8,1
...,...,...,...,...,...,...,...,...
101,2021-12-12 23:32:58+00:00,1470174872227700737,decisions 🤔 #doge #dogecoin #DoOnlyGoodEveryda...,MilgateTyler,108,32,14,2
102,2021-12-12 22:50:34+00:00,1470164202404777990,I didn’t buy #dogecoin off hype. I didn’t buy ...,DogeDillionaire,132,25,19,3
103,2021-12-12 22:23:59+00:00,1470157510522908673,#dogecoin #doge $Doge #DogecoinToTheMoon 🌑🌌 h...,YoungCrypto69,260,81,43,7
104,2021-12-12 21:52:50+00:00,1470149670085316612,We are currently FLYING through our last #NFTs...,CheemsInu,440,213,98,25


In [29]:
EraseFiles(r"\new_scraped_tweets_*.parquet")

Aca se va a chequear qué porcentaje de los dataframes se superpone y se duplican los datos. Esto es importante porque esto es tiempo gastado en procesar datos que ya se tienen. 
Es posible que esto pase cuando las particiones que se realizan quedan con poca cantidad de tweets cada uno y hay mas chances de que de una iteración a otra no se cambie de dia asignado a 
"fecha_menor".


Con 50.000 tweets y 10 particiones- el tiempo requerido fue de 41,9min y el porcentaje de duplicados fue de 17%


In [25]:
#Check for overlapping between dataframes
percentage = dafatame_vertical_stack.duplicated().sum()/len(dafatame_vertical_stack)
print("Percentaje of duplicated rows:",percentage)


Percentaje of duplicated rows: 0.17036450079239301


In [27]:
#Re check after dropping duplicates
percentage = dafatame_vertical_stack_no_duplicated.duplicated().sum()/len(dafatame_vertical_stack_no_duplicated)
print("Percentaje of duplicated rows:",percentage)

Percentaje of duplicated rows: 0.0


In [28]:
dafatame_vertical_stack_no_duplicated

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-19 23:58:11+00:00,1472717932916576259,Keep up the good work Elon @elonmusk ⚒️🔥 Show ...,DogecoinNorway,607,100,38,6
1,2021-12-19 23:36:20+00:00,1472712433869279232,Congrats on 900k followers @BillyM2k! (Billy c...,itsALLrisky,313,47,19,1
2,2021-12-19 23:11:11+00:00,1472706104991879171,#Dogecoin will make millionaires,itsALLrisky,808,109,81,6
3,2021-12-19 23:08:01+00:00,1472705306182492164,Congratulations to #DogeBonk for becoming the ...,kcsdaily,428,220,94,11
4,2021-12-19 22:40:10+00:00,1472698297521348614,#SORRYELON WE MADE IT TO OUTER SPACE BEFORE #D...,bonktovi,107,43,8,1
...,...,...,...,...,...,...,...,...
71,2021-12-03 00:19:35+00:00,1466562722905034752,Elon Musk just liked a tweet from his x-girlfr...,MattWallace888,3678,448,273,9
72,2021-12-02 23:42:50+00:00,1466553476071129089,Hurry grab your bag we getting ready for the m...,TaxAveryShow,120,33,39,4
73,2021-12-02 23:29:41+00:00,1466550164487819264,Twitter Post Protest - Day 16\n #ElonMusk #Dog...,CryptoThro,114,44,16,0
74,2021-12-02 23:13:41+00:00,1466546138715246593,Use your brain and buy some $DOGE! #dogecoin,usssks,387,58,32,1


In [61]:
dataframe = GetTweets(1000,500)

time elpased: 0.8 min


In [62]:
dataframe

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-19 23:58:11+00:00,1472717932916576259,Keep up the good work Elon @elonmusk ⚒️🔥 Show ...,DogecoinNorway,611,101,38,6
1,2021-12-19 23:11:11+00:00,1472706104991879171,#Dogecoin will make millionaires,itsALLrisky,809,109,81,6
2,2021-12-19 22:26:06+00:00,1472694757277122561,Wow #Dogecoin $DOGE https://t.co/AAqCvTnmUF,itsALLrisky,698,109,54,6
3,2021-12-19 22:01:06+00:00,1472688467083681794,Dogebonk #DOBO has beaten #ElonMusk to achieve...,BMcGleenon,656,269,239,26
4,2021-12-19 21:30:03+00:00,1472680653858787330,@elonmusk The most important thing to teach ki...,MattWallace888,622,57,131,3
5,2021-12-19 21:19:10+00:00,1472677916039331849,Are any of my followers actual real people tha...,usssks,665,63,124,7
6,2021-12-19 19:01:43+00:00,1472643323487731721,The market capitalization of meme-based digita...,BTCTN,1110,288,246,13
7,2021-12-19 18:29:19+00:00,1472635169161060356,HODL Starting To Trend!! LET’S MAKE IT GO CRAZ...,MattWallace888,1789,389,222,4


In [13]:
tweets_df2

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-19 23:58:11+00:00,1472717932916576259,Keep up the good work Elon @elonmusk ⚒️🔥 Show ...,DogecoinNorway,611,101,38,6
1,2021-12-19 23:36:20+00:00,1472712433869279232,Congrats on 900k followers @BillyM2k! (Billy c...,itsALLrisky,313,48,19,1
2,2021-12-19 23:11:11+00:00,1472706104991879171,#Dogecoin will make millionaires,itsALLrisky,809,109,81,6
3,2021-12-19 23:08:01+00:00,1472705306182492164,Congratulations to #DogeBonk for becoming the ...,kcsdaily,431,224,96,12
4,2021-12-19 22:40:10+00:00,1472698297521348614,#SORRYELON WE MADE IT TO OUTER SPACE BEFORE #D...,bonktovi,107,43,8,1
...,...,...,...,...,...,...,...,...
97,2021-12-18 18:45:35+00:00,1472276874437054464,#dogecoin will recover!,MYLESGINVEST,195,21,21,0
98,2021-12-18 18:37:02+00:00,1472274722968670208,"I’m holding #dogecoin ,doge father will take ...",Adogeworld,215,57,39,2
99,2021-12-18 18:10:46+00:00,1472268113316139020,Diamond hands are going to the moon when #Doge...,DianaKFletcher,913,136,158,4
100,2021-12-18 18:10:06+00:00,1472267947662282752,I found this #dogecoin 🥇✋🏼💎🤚🏼 https://t.co/V1H...,dogesuppporteam,180,41,28,2


In [14]:
tweets_df3

,Datetime,Tweet Id,Text,Username,Likes,Retweet,Reply,Quote
0,2021-12-19 23:58:11+00:00,1472717932916576259,Keep up the good work Elon @elonmusk ⚒️🔥 Show ...,DogecoinNorway,611,101,38,6
1,2021-12-19 23:36:20+00:00,1472712433869279232,Congrats on 900k followers @BillyM2k! (Billy c...,itsALLrisky,313,48,19,1
2,2021-12-19 23:11:11+00:00,1472706104991879171,#Dogecoin will make millionaires,itsALLrisky,809,109,81,6
3,2021-12-19 23:08:01+00:00,1472705306182492164,Congratulations to #DogeBonk for becoming the ...,kcsdaily,431,224,96,12
4,2021-12-19 22:40:10+00:00,1472698297521348614,#SORRYELON WE MADE IT TO OUTER SPACE BEFORE #D...,bonktovi,107,43,8,1
...,...,...,...,...,...,...,...,...
170,2021-12-17 14:22:00+00:00,1471848154186268672,To The Moon And Beyond \n#Dogecoin #DogecoinTo...,DogeMoon11,133,32,19,1
171,2021-12-17 14:00:12+00:00,1471842670083559438,Which one do you like the most? 🧐 \n\n#BabyTK ...,ShibaArchives,279,252,110,2
172,2021-12-17 13:54:29+00:00,1471841231315550215,My moms neighbors are bullish af!! \n#dogearmy...,TheUnderDoge3,251,28,42,0
173,2021-12-17 13:01:08+00:00,1471827804811677701,Fate loves Irony 🐶 #dogecoin #doge #tesla @elo...,DogecoinNorway,124,39,14,0


In [6]:
tweets_list3 = []

# Scrapeando los tweets que contengan #doge, con since y until se le indica el período
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:elonmusk').get_items()):
    if tweet.likeCount > 1000:
        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount ])
    if i>20000:
        break
        
tweets_df3 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])

KeyboardInterrupt: 

In [ ]:
tweets_df3

In [72]:
l=[]
for i,t in enumerate(tweets_df3.Text):
    if(t.find('#Dogecoin')!=-1):
        l.append([i,t])
elon_df = pd.DataFrame(l, columns=['Index','Text'])

In [75]:
elon_df

,Index,Text
0,0,The market capitalization of meme-based digita...
1,1,HODL Starting To Trend!! LET’S MAKE IT GO CRAZ...
2,2,Which of these will happen first in 2022? \n\n...
3,3,"On this day one year ago\n\n1 #Bitcoin: $23,13..."
4,4,"On this day one year ago,\n\n#Bitcoin: $23,132..."


In [76]:
string = tweets_df3.loc[1000,"Text"]

In [77]:
string.find("#Dogecoin")!=-1

False

In [52]:
for i in elon_df.Text:
    print(i)

In [ ]:
%%time
import snscrape.modules.twitter as sntwitter
import pandas as pd

# lista donde se guardaran los tweets y su info
tweets_list2 = []

# Scrapeando los tweets que contengan #doge, con since y until se le indica el período
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#doge since:2021-05-31 until:2021-12-20').get_items()):
    if i>500000:
        break
# Con append agrego en cada iteracion el tweet y sus metricas a la lista
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount])
    
# Creo el DataFrame respetando el orden en el que se guardo cada campo en la lista anterior
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])
tweets_df2.to_parquet("scrapped_tweets_future.parquet")

In [ ]:
%%time
import snscrape.modules.twitter as sntwitter
import pandas as pd

# lista donde se guardaran los tweets y su info
tweets_list2 = []

# Scrapeando los tweets que contengan #doge, con since y until se le indica el período
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#doge since:2021-05-31 until:2021-12-22').get_items()):
    j = i+1
    if j >=100:
        tweets_df2.to_parquet("scrapped_tweets_future.parquet")
        j=0
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount])
    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])
            
    if i>10000:
         break
               
# Con append agrego en cada iteracion el tweet y sus metricas a la lista

    
    
# Creo el DataFrame respetando el orden en el que se guardo cada campo en la lista anterior
#tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])

In [ ]:
#Muestro los tweets con mayor numero de likes
tweets_df2.sort_values(by = 'Likes', ascending = False)

In [ ]:
tweets_df2.head()

In [ ]:
tweets_df2.tail()

In [ ]:
from datetime import datetime, timedelta



In [ ]:

tweets_df2["Dates"] = pd.to_datetime(tweets_df2['Datetime']).dt.time

In [ ]:
tweets_df2["Dates"]

In [ ]:
#last_date_index
str(tweets_df2.loc[tweets_df2.index.stop-1,"Dates"])

In [ ]:
str(tweets_df2.loc[tweets_df2.index.stop-1,"Tweet Id"])

In [ ]:
#Creo el parquet
tweets_df2.to_parquet('tweets.parquet')

In [ ]:
tweets_df2.index.start

In [ ]:
def Generar_tweets_db(dataframe, cantidad_a_traer):
    tweets_list2 = []
    dataframe["Dates"] = pd.to_datetime(dataframe['Datetime']).dt.date
    last_date = str(dataframe.loc[dataframe.index.stop-1,"Dates"]+ timedelta(days = 1)) 
    first_id = str(dataframe.loc[dataframe.index.stop-1,"Tweet Id"])
# Scrapeando los tweets que contengan #doge, con since y until se le indica el período
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#doge since:2021-01-01 until:'+last_date).get_items()):
        if i>cantidad_a_traer:
            break
# Con append agrego en cada iteracion el tweet y sus metricas a la lista
        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount])
    
# Creo el DataFrame respetando el orden en el que se guardo cada campo en la lista anterior
    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])
    tweets_df2.to_parquet("scrapped_tweets_{id_tweet}.parquet".format(id_tweet = first_id))

In [ ]:
def Generar_tweets_db_2(dataframe, cantidad_a_traer):
    tweets_list2 = []
    dataframe["Dates"] = pd.to_datetime(dataframe['Datetime']).dt.date
    last_date = str(dataframe.loc[dataframe.index.stop-1,"Dates"]+ timedelta(days = 1)) 
    first_id = str(dataframe.loc[dataframe.index.stop-1,"Tweet Id"])
# Scrapeando los tweets que contengan #doge, con since y until se le indica el período
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#doge since:2021-05-31 until:'+last_date).get_items()):
        if i>cantidad_a_traer:
            break
# Con append agrego en cada iteracion el tweet y sus metricas a la lista
        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.replyCount, tweet.quoteCount])
    
# Creo el DataFrame respetando el orden en el que se guardo cada campo en la lista anterior
    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes','Retweet','Reply','Quote'])
    tweets_df2.to_parquet("scrapped_tweets_{id_tweet}.parquet".format(id_tweet = first_id))

In [ ]:
tweets_scrapeados = pd.read_parquet("scrapped_tweets.parquet")

In [ ]:
tweets_scrapeados.head()

In [ ]:
tweets_scrapeados.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados, 10000)

In [ ]:
tweets_scrapeados2 = pd.read_parquet("scrapped_tweets_1398701696726011908.parquet")

In [ ]:
tweets_scrapeados2.loc[tweets_scrapeados2.index.start,"Dates"]

In [ ]:
tweets_scrapeados2 = pd.read_parquet("scrapped_tweets_1398701696726011908.parquet")

In [ ]:
Generar_tweets_db(tweets_scrapeados2, 10000)

In [ ]:
tweets_scrapeados2.tail()

In [ ]:
tweets_scrapeados3 = pd.read_parquet("scrapped_tweets_1398346764692955136.parquet")

In [ ]:
tweets_scrapeados3.head()

In [ ]:
tweets_scrapeados3.tail()

In [ ]:
%%time
Generar_tweets_db(tweets_scrapeados3, 10000)

In [ ]:
tweets_scrapeados4 = pd.read_parquet("scrapped_tweets_1398119625838055426.parquet")

In [ ]:
tweets_scrapeados4.head()

In [ ]:
tweets_scrapeados4.tail()

In [ ]:
%%time
Generar_tweets_db(tweets_scrapeados4, 500000)

In [ ]:
tweets_scrapeados4 = pd.read_parquet("scrapped_tweets_1398119963651579908.parquet")

In [ ]:
tweets_scrapeados4.head()

In [ ]:
tweets_scrapeados4.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados4, 500000)

In [ ]:
tweets_scrapeados5 = pd.read_parquet("scrapped_tweets_1389258164986650629.parquet")


In [ ]:
tweets_scrapeados5.head()

In [ ]:
tweets_scrapeados5.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados5, 500000)

In [ ]:
tweets_scrapeados6 = pd.read_parquet("scrapped_tweets_1368829591532503040.parquet")

In [ ]:
tweets_scrapeados6.head()

In [ ]:
tweets_scrapeados6.tail()

In [ ]:
tweets_scrapeados7 = pd.read_parquet("scrapped_tweets_future.parquet")

In [ ]:
tweets_scrapeados7.head()

In [ ]:
tweets_scrapeados7.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados7, 500000)

In [ ]:
tweets_scrapeados7 = pd.read_parquet("scrapped_tweets_1447133836261957632.parquet")

In [ ]:
tweets_scrapeados7.head()

In [ ]:
tweets_scrapeados7.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados7, 500000)

In [ ]:
tweets_scrapeados8 = pd.read_parquet("scrapped_tweets_1412704015494922240.parquet")

In [ ]:
tweets_scrapeados8.head()

In [ ]:
tweets_scrapeados8.tail()

In [ ]:
Generar_tweets_db(tweets_scrapeados8, 500000)

In [ ]:
tweets_scrapeados9 = pd.read_parquet("scrapped_tweets_1393317715587305472.parquet")

In [ ]:
tweets_scrapeados9.head()

In [ ]:
tweets_scrapeados9.tail()